### pCrunch Update

Jake Nunemaker

Last Updated: 01/03/2023 (gbarter)

In [1]:
import os
from fnmatch import fnmatch

import numpy as np
import pandas as pd
import ruamel.yaml as ry

from pCrunch import LoadsAnalysis, PowerProduction, FatigueParams
from pCrunch.io import load_FAST_out
from pCrunch.utility import save_yaml, get_windspeeds, convert_summary_stats

def valid_extension(fp):
    return any([fnmatch(fp, ext) for ext in ["*.outb", "*.out"]])

#### Project Directory

In [2]:
output_dir = "/Users/gbarter/devel/pCrunch/tests/io/data"
results_dir = os.path.join(output_dir, "results")
save_results = True

outfiles = [
    os.path.join(output_dir, f) for f in os.listdir(output_dir)
    if valid_extension(f)
]

print(f"Found {len(outfiles)} files.")

Found 9 files.


#### Interacting with output files

In [3]:
# The new framework provides an object oriented framework to interact with
# output files. The easiest way to use this is to use the 'load_FAST_out' function.

outputs = load_FAST_out(outfiles[:3])
outputs

In [4]:
# An instance of 'OpenFASTBinary' (or 'OpenFASTAscii' if applicable) is created.
# The instance stores the raw data but also provides many useful methods for
# interacting with the data:

# print(outputs[0].data)
# print(outputs[0].time)
print(outputs[0].channels)
# print(outputs[0].maxima)
# print(outputs[0].stddevs)

# Individual channel time series can also be accessed with dict style indexing:
#outputs[0]["Wind1VelX"]

['Time' 'WindVxi' 'WindVyi' 'WindVzi' 'WaveElev' 'Wave1Vxi' 'Wave1Vyi'
 'Wave1Vzi' 'Wave1Axi' 'Wave1Ayi' 'Wave1Azi' 'GenPwr' 'GenTq' 'HSSBrTq'
 'BldPitch1' 'BldPitch2' 'BldPitch3' 'Azimuth' 'RotSpeed' 'GenSpeed'
 'NacYaw' 'NacYawErr' 'OoPDefl1' 'IPDefl1' 'TwstDefl1' 'OoPDefl2'
 'IPDefl2' 'TwstDefl2' 'OoPDefl3' 'IPDefl3' 'TwstDefl3' 'TwrClrnc1'
 'TwrClrnc2' 'TwrClrnc3' 'NcIMUTAxs' 'NcIMUTAys' 'NcIMUTAzs' 'TTDspFA'
 'TTDspSS' 'TTDspTwst' 'PtfmSurge' 'PtfmSway' 'PtfmHeave' 'PtfmRoll'
 'PtfmPitch' 'PtfmYaw' 'PtfmTAxt' 'PtfmTAyt' 'PtfmTAzt' 'RootFxc1'
 'RootFyc1' 'RootFzc1' 'RootMxc1' 'RootMyc1' 'RootMzc1' 'RootFxc2'
 'RootFyc2' 'RootFzc2' 'RootMxc2' 'RootMyc2' 'RootMzc2' 'RootFxc3'
 'RootFyc3' 'RootFzc3' 'RootMxc3' 'RootMyc3' 'RootMzc3' 'Spn1MLxb1'
 'Spn1MLyb1' 'Spn1MLzb1' 'Spn1MLxb2' 'Spn1MLyb2' 'Spn1MLzb2' 'Spn1MLxb3'
 'Spn1MLyb3' 'Spn1MLzb3' 'RotThrust' 'LSSGagFya' 'LSSGagFza' 'RotTorq'
 'LSSGagMya' 'LSSGagMza' 'YawBrFxp' 'YawBrFyp' 'YawBrFzp' 'YawBrMxp'
 'YawBrMyp' 'YawBrMzp' 'TwrBsFxt

#### pCrunch Configuration

In [5]:
# Channel magnitudes are defined in a dict:
magnitude_channels = {
    "RootMc1": ["RootMxc1", "RootMyc1", "RootMzc1"],
    "RootMc2": ["RootMxc2", "RootMyc2", "RootMzc2"],
    "RootMc3": ["RootMxc3", "RootMyc3", "RootMzc3"],
}

# Define channels (and their fatigue slopes) in a dict:
fatigue_channels = {
    "RootMc1": FatigueParams(lifetime=25.0, slope=10.0, ult_stress=6e8),
    "RootMc2": FatigueParams(lifetime=25.0, slope=10.0, ult_stress=6e8),
    "RootMc3": FatigueParams(lifetime=25.0, slope=10.0, ult_stress=6e8),
}

# Define channels to save extreme data in a list:
channel_extremes = [
    "RotSpeed",
    "RotThrust",
    "RotTorq",
    "RootMc1",
    "RootMc2",
    "RootMc3",
]

#### Run pCrunch

In [6]:
# The API has changed and is in more of an object oriented framework.
la = LoadsAnalysis(
    outfiles[:5],                           # The primary input is a list of output files
    magnitude_channels=magnitude_channels,  # All of the following inputs are optional
    fatigue_channels=fatigue_channels,      # 
    extreme_channels=channel_extremes,      #
    trim_data=(0,),                         # If 'trim_data' is passed, all input files will
)                                           # be trimmed to (tmin, tmax(optional))

la.process_outputs(cores=4)                 # Once LoadsAnalysis is configured, process outputs with
                                            # `process_outputs`. `cores` is optional but will trigger parallel processing if configured

#### Outputs

In [7]:
# The summary stats per each file are here:
la.summary_stats

,WindVxi,WindVyi,WindVzi,WaveElev,Wave1Vxi,Wave1Vyi,Wave1Vzi,Wave1Axi,Wave1Ayi,Wave1Azi,...,RootMFlp3,Spn4MLxb1,Spn4MLyb1,LSSGagFxs,LSSGagFys,LSSGagFzs,LSShftTq,HSShftTq,LSShftPwr,HSShftPwr
,min,min,min,min,min,min,min,min,min,min,...,integrated,integrated,integrated,integrated,integrated,integrated,integrated,integrated,integrated,integrated
Test1.outb,4.304633,-2.989339,-1.865983,-1.104790,-1.119737,0.0,-1.141634,-1.313537,0.0,-1.325566,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Test2.outb,6.610294,-3.429822,-2.825261,-1.490053,-1.520973,0.0,-1.875418,-2.522711,0.0,-2.531182,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
DLC2.3_3.out,8.197000,0.000000,0.000000,-1.135000,-0.540300,0.0,-0.515600,-0.248200,0.0,-0.183600,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
step_0.outb,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
AOC_WSt.outb,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,-20.908183,1.641742,-5.823535,178.350302,-0.061129,-200.13404,75.635627,2.677367,709.550948,709.430941


In [8]:
# These are indexable by channel, stat:
la.summary_stats["RootMc1"]

,min,max,std,mean,median,abs,integrated
Test1.outb,2108.201968,12256.759434,1605.138048,6479.421752,6327.050655,12256.759434,3.887742e+06
Test2.outb,2880.976909,14168.307178,1813.222085,8773.555158,8798.187968,14168.307178,5.264308e+06
DLC2.3_3.out,347.604352,8986.223847,2657.649371,5099.936575,6449.890708,8986.223847,3.060075e+05
step_0.outb,9549.757719,55478.912370,6376.015420,36717.478050,35928.506566,55478.912370,3.672081e+06
AOC_WSt.outb,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [9]:
la.summary_stats[("RootMc1", 'min')]

Test1.outb      2108.201968
Test2.outb      2880.976909
DLC2.3_3.out     347.604352
step_0.outb     9549.757719
AOC_WSt.outb            NaN
Name: (RootMc1, min), dtype: float64

In [10]:
# Or by file
la.summary_stats.loc["DLC2.3_3.out"]

WindVxi    min           8.1970
WindVyi    min           0.0000
WindVzi    min           0.0000
WaveElev   min          -1.1350
Wave1Vxi   min          -0.5403
                          ...  
LSSGagFzs  integrated       NaN
LSShftTq   integrated       NaN
HSShftTq   integrated       NaN
LSShftPwr  integrated       NaN
HSShftPwr  integrated       NaN
Name: DLC2.3_3.out, Length: 1799, dtype: float64

In [11]:
# Damage equivalent loads are found here:
la.DELs

,RootMc1,RootMc2,RootMc3
Test1.outb,5372.081676,5487.951352,5267.372170
step_0.outb,28836.838406,16210.632541,33824.200074
Test2.outb,6372.006628,6484.887751,6548.485829
DLC2.3_3.out,5708.372415,5607.452586,2132.788161
AOC_WSt.outb,0.003558,0.003558,0.003558


In [12]:
# Extreme events:
la.extreme_events

{'RotSpeed': [{'Time': 273.4000031799078,
   'RotSpeed': 11.55297565460205,
   'RotThrust': 745.5087280273438,
   'RotTorq': 3122.724365234375,
   'RootMc1': 9590.92475660036,
   'RootMc2': 8392.58661304791,
   'RootMc3': 9467.578392491985},
  {'Time': 8.0,
   'RotSpeed': 7.361658573150635,
   'RotThrust': 2297.14599609375,
   'RotTorq': 13777.115234375,
   'RootMc1': 41062.48033129582,
   'RootMc2': 49675.9908595564,
   'RootMc3': 48690.10941389519},
  {'Time': 463.1000060066581,
   'RotSpeed': 14.304065704345703,
   'RotThrust': 725.6522827148438,
   'RotTorq': 4229.85791015625,
   'RootMc1': 7988.654888226473,
   'RootMc2': 7507.5203383009475,
   'RootMc3': 9389.899303100416},
  {'Time': 61.9,
   'RotSpeed': 11.33,
   'RotThrust': 317.0,
   'RotTorq': 140.8,
   'RootMc1': 2092.704239399347,
   'RootMc2': 3457.296428656357,
   'RootMc3': 3844.6280631681398},
  {'Time': 35.0,
   'RotSpeed': 109.06758293648133,
   'RootMc1': nan,
   'RootMc2': nan,
   'RootMc3': nan}],
 'RotThrust': [{